In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "dw"
dw__thr = 13
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

In [3]:
dot_image_folder_path = f"/mnt/data/Imaging/202105-Deconwolf/data_210726/{selected_magnification}_{selected_image_type}"
ref_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw"
dot_mask_folder_path = "../../data/20x_mask/dilated_labels_watershed_from60x"
ref_mask_folder_path = "../../data/60x_mask/dilated_labels_watershed"

### Read shift data

In [4]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [5]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [6]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.tsv", sep="\t")

In [7]:
match_output: List[np.ndarray] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    dot_max_z_proj = tifffile.imread(os.path.join(dot_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    ref_max_z_proj = tifffile.imread(os.path.join(ref_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    
    dot_labels = tifffile.imread(os.path.join(dot_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.from_60x.tiff")).reshape(dot_max_z_proj.shape)
    ref_labels = tifffile.imread(os.path.join(ref_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.tiff")).reshape(ref_max_z_proj.shape)
    
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[dw__thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy()/ aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    pdist = cdist(dot_coords2, ref_coords)
    
    matched: List[Tuple[int, int, float]] = []
    dot_id = 0
    while dot_id < pdist.shape[0]:
        ref_id = np.nanargmin(pdist[dot_id, :])
        if np.nanargmin(pdist[:, ref_id]) == dot_id:
            matched.append((dot_id, ref_id, pdist[dot_id, ref_id]))
            pdist[dot_id, :] = np.nan
            pdist[:, ref_id] = np.nan
        dot_id += 1

    matched_a = np.array(matched)
    match_output.append(pd.DataFrame(dict(
        series=current_field_id,
        dot_id=matched_a[:, 0].astype("i"),
        ref_id=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    )))
    match_counts.append((current_field_id, matched_a.shape[0], dot_coords.shape[0], ref_coords.shape[0]))
    
    fig3, ax = plt.subplots(figsize=(30, 10), ncols=3, constrained_layout=True)
    fig3.suptitle(f"Field #{current_field_id}")
    print(" > Plotting dot")
    ax[0].set_title(f"{selected_magnification}_{selected_image_type}")
    ax[0].imshow(
        dot_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=dot_max_z_proj.min(), vmax=dot_max_z_proj.max(),
        resample=False, filternorm=False)
    ax[0].scatter(
        x=dot_coords["y"].values,
        y=dot_coords["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting ref")
    ax[1].set_title("60x_dw")
    ax[1].imshow(
        ref_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=ref_max_z_proj.min()*1.5, vmax=ref_max_z_proj.max()*.5,
        resample=False, filternorm=False)
    ax[1].scatter(
        x=ref_coords["y"].values,
        y=ref_coords["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting contours [ref]")
    for lid in range(1, dot_labels.max()):
        contours = measure.find_contours(dot_labels == lid, 0.8)
        for contour in contours:
            ax[0].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    print(" > Plotting contours [dot]")
    for lid in range(1, ref_labels.max()):
        contours = measure.find_contours(ref_labels == lid, 0.8)
        for contour in contours:
            ax[1].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    print(" > Plotting overlapped points between raw and dw")
    ax[2].set_title(f"Red: {selected_magnification}_{selected_image_type}. Blue: 60x_dw.")
    ax[2].plot(
        dot_coords["y"].values,
        dot_coords["x"].values,
        'r.', marker=".", markersize=2)
    ax[2].plot(
        ref_coords["y"].values,
        ref_coords["x"].values,
        'b.', marker=".", markersize=.8)
    plt.close(fig3)
    print(" > Exporting")
    fig3.savefig(os.path.join("../../data/magnifications_matching",
                              f"{selected_magnification}_{selected_image_type}_overlays_tentative_thr",
                              f"overlay_{current_field_id:03d}.png"), bbox_inches='tight')
    print(" ! DONE")

  0%|          | 0/7 [00:00<?, ?it/s]

 > Plotting dot
 > Plotting ref
 > Plotting contours [ref]
 > Plotting contours [dot]
 > Plotting overlapped points between raw and dw
 > Exporting


 14%|█▍        | 1/7 [00:16<01:38, 16.41s/it]

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [ref]
 > Plotting contours [dot]
 > Plotting overlapped points between raw and dw
 > Exporting


 29%|██▊       | 2/7 [00:36<01:27, 17.48s/it]

...

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [ref]
 > Plotting contours [dot]
 > Plotting overlapped points between raw and dw
 > Exporting


 86%|████████▌ | 6/7 [01:51<00:19, 19.24s/it]

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [ref]
 > Plotting contours [dot]
 > Plotting overlapped points between raw and dw
 > Exporting


100%|██████████| 7/7 [02:12<00:00, 18.91s/it]

 ! DONE


In [8]:
match_counts_a = np.array(match_counts)
p_raw = match_counts_a.sum(0)[1]/match_counts_a.sum(0)[2]*100
p__dw = match_counts_a.sum(0)[1]/match_counts_a.sum(0)[3]*100
print(f"{p_raw:.2f}% 20x dots matched to 60x")
print(f"{p__dw:.2f}% 60x dots matched to 20x")

80.62% 20x dots matched to 60x
70.06% 60x dots matched to 20x


In [9]:
for i in range(match_counts_a.shape[0]):
    p_raw = match_counts_a[i, 1]/match_counts_a[i, 2]*100
    p__dw = match_counts_a[i, 1]/match_counts_a[i, 3]*100
    print(f"[Field #{i+1}] {p_raw:.2f}% 20x dots matched to 60x")
    print(f"[Field #{i+1}] {p__dw:.2f}% 60x dots matched to 20x")
    print("---")

[Field #1] 78.51% 20x dots matched to 60x
[Field #1] 73.91% 60x dots matched to 20x
---
[Field #2] 81.19% 20x dots matched to 60x
[Field #2] 74.11% 60x dots matched to 20x
---
[Field #3] 80.89% 20x dots matched to 60x
[Field #3] 70.40% 60x dots matched to 20x
---
[Field #4] 83.72% 20x dots matched to 60x
[Field #4] 76.18% 60x dots matched to 20x
---
[Field #5] 81.78% 20x dots matched to 60x
[Field #5] 70.33% 60x dots matched to 20x
---
[Field #6] 76.48% 20x dots matched to 60x
[Field #6] 63.68% 60x dots matched to 20x
---
[Field #7] 69.38% 20x dots matched to 60x
[Field #7] 34.62% 60x dots matched to 20x
---
